# Situation and Problem 

Some businesses tend to be quite influenced by how commodity market behaves, particularly for the ones that rely on it as a primary input, such as mills, flour factories. However, if managing it properly, we can have not only properly manage the risk, but perhaps profit from the trading a bit. To that end, how to properly manage the purchase and sell scheudule is critcal. This small project demonstrates how to leverage linear programming to optimize for the best purchase schedule, given a set of buy and sell prices for the next a little while (less than or up to a year).

## Environment Check

In [ ]:
import datetime
print(datetime.datetime.now())

2022-06-05 19:37:35.089597


In [ ]:
from platform import python_version
print(python_version())

3.7.13


To start with, we are given a series of buy and sell prices for a commodity (wheat in this case) for the next 10 months, subject to a couple of constraints:
- the warehouse can only store 20000 bushels
- in any given month, we can only buy certain amount of wheat, up to warehouse size
-  in any given month, we can only sell certain amount of wheat, up to what we have at the beginning of the month

## Install and Import Packages

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 905 kB/s 
     |████████████████████████████████| 6.4 MB 11.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 4.7 MB/s 


In [ ]:
import pulp as pl
import pandas as pd
import numpy as np
import re

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import yfinance as yf
from collections import OrderedDict

from IPython.core.interactiveshell import InteractiveShell
import warnings

In [ ]:
# Pandas version check and settings
print('The pandas version is {}.'.format(pd.__version__))
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)
pd.options.plotting.backend = "plotly"

The pandas version is 1.3.5.


In [ ]:
# Jupyter notebooks settigns 
InteractiveShell.ast_node_interactivity = "all" #allow multiple outputs per executed cell
warnings.filterwarnings('ignore') #surpress warnings

In [ ]:
# matplot/Plot Settings

%matplotlib inline 
plt.rcParams['figure.figsize'] = (25, 25) #Change sizes for all figures

In [ ]:
# pulp setup

solver_list = pl.listSolvers(onlyAvailable=True)
solver_list

['PULP_CBC_CMD']

## Set up the context 

In this case, we use one of the most quoted wheat future contracts available in US, Kansis City (KC) Wheat Contract, Hard Red Winter Wheat

Quotes are from the most active futures exchange in North America, CME Group, retrieved from Yahoo Finance

Values are retrieved from Yahoo Finance.

The contract delivers 5 times a year has the name fashion of KEH[YY], KEK[YY], KEN[YY], KEU[YY], KEZ[YY], as March, May, Jul, Sept, Dec

Active contracts are listed for the next 15 months, so a 2-year time frame should more than suffice.

Further, here are some of the conditions that we are bringing in:
- there is a series of demands of wheat from the mill/factory will process, subject to human inputs and decisions; here we also assume that demand cannot exceed the warehouse size
- the wheat-buying amounts need to accomodate the demand, making sure that together with the beginning inventory of the month, we can satisfy the need of the period before next available futures contract available on the market
- the wheat-buying amount cannot exceed warehouse size or we don't have place to hold the bushels! 
- the amount of wheat to sell cannot exceed wthe beginning inventory during that period.
- we cannot have a ending inventory balance greater than warehouse size
- holding cost = 10% of purchase price of that period multiply by the beginning inventory of the month

## Get Real Data

In some cases, raw pricing quotes retrieved are not quite useable, therefore,we implement pricing data processing rules:
1. if all are NaN then drop the columns/months
2. if selling price = NaN, then assuming 10% off from the purchase price of the month
3. if purchase price = NaN, then assuming 10% off from the selling price of the month

In [ ]:
def get_commodity_bid_ask_prices(starting_year, generic_ticker_list, delivery_month_list, contract_size, default_bid_ask_discount):
  # variables to be defined to generate proper futures tickers, ticker_list and deliver_month must be synced and chronologically sequencial 
  generic_ticker_list=["KEH", "KEK", "KEN", "KEU", "KEZ"]
  delivery_month_list= ["March", "May", "July", "September", "December"]
  year_list = list(range(starting_year, starting_year+3, 1))
  actual_delivery_month_list = [f'{str(year)}_{month}' for year in year_list for month in delivery_month_list]
  futures_ticker_list = [f'{ticker}{str(year)[-2:]}.CBT' for year in year_list for ticker in generic_ticker_list]

  purchase_price_list = []
  for ticker in futures_ticker_list:
    try: 
      yf.Ticker(ticker).info['ask']
    except: 
      purchase_price_list.append(np.nan)
    else:
      purchase_price_list.append(round(yf.Ticker(ticker).info['ask'], 2))
  
  selling_price_list = []
  for ticker in futures_ticker_list:
    try: 
      yf.Ticker(ticker).info['bid']
    except: 
      selling_price_list.append(np.nan)
    else:
      selling_price_list.append(round(yf.Ticker(ticker).info['bid'],2))
  
  # data processing rules:
  # 1. if all are NaN then drop the columns/months
  # 2. if selling price = NaN, then assuming 10% off from the purchase price of the month
  # 3. if purchase price = NaN, then assuming 10% off from the selling price of the month
  
  commodity_price_data = {'Selling Price': selling_price_list, 'Purchase Price': purchase_price_list}
  commodity_price = pd.DataFrame.from_dict(commodity_price_data, orient= "index", columns = actual_delivery_month_list)
  commodity_price = commodity_price.replace({0: np.NaN}).dropna(axis=1, how="all")
  for x in commodity_price.columns:
    if commodity_price[(commodity_price[x].isnull())].index == 'Purchase Price':
      commodity_price.loc['Purchase Price', x] = round(commodity_price.loc['Selling Price', x] * (1+default_bid_ask_discount),2)
    elif commodity_price[(commodity_price[x].isnull())].index == 'Selling Price':
      commodity_price.loc['Selling Price', x] = round(commodity_price.loc['Purchase Price', x] * (1-default_bid_ask_discount),2)
    else:
      pass
  commodity_price = commodity_price.div(contract_size)

  return futures_ticker_list, commodity_price

In [ ]:
ticker_list, commodity_price_table = get_commodity_bid_ask_prices(starting_year=2022, 
                                                                  generic_ticker_list=["KEH", "KEK", "KEN", "KEU", "KEZ"], 
                                                                  delivery_month_list= ["March", "May", "July", "September", "December"], 
                                                                  contract_size=5000, 
                                                                  default_bid_ask_discount=0.1)

ticker_list
commodity_price_table

['KEH22.CBT',
 'KEK22.CBT',
 'KEN22.CBT',
 'KEU22.CBT',
 'KEZ22.CBT',
 'KEH23.CBT',
 'KEK23.CBT',
 'KEN23.CBT',
 'KEU23.CBT',
 'KEZ23.CBT',
 'KEH24.CBT',
 'KEK24.CBT',
 'KEN24.CBT',
 'KEU24.CBT',
 'KEZ24.CBT']

,2022_July,2022_September,2022_December,2023_March,2023_May,2023_July,2023_September,2023_December,2024_March,2024_July
Selling Price,0.22325,0.2253,0.2259,0.2284,0.2220,0.210,0.1536,0.20305,0.18378,0.1622
Purchase Price,0.22560,0.2287,0.2356,0.2590,0.2388,0.222,0.2250,0.22710,0.20420,0.1837


## Modelling

In [ ]:
def setup_optimization_model(warehouse_size, beginning_inventory, commodity_name, commodity_price_table, demand, holding_cost_as_percent_to_purchase_price):
  commodity_buy_sell_model = pl.LpProblem(name="Commodity_Supply_Management_Schedule", sense=pl.LpMaximize)

  # Decision Variables
  commodity_to_buy_list = [pl.LpVariable(name=f'{commodity_name}_to_buy_{str(x)}', lowBound=0) for x in commodity_price_table.columns]
  commodity_to_sell_list = [pl.LpVariable(name=f'{commodity_name}_to_sell_{str(x)}', lowBound=0) for x in commodity_price_table.columns]

  # set up ending inventory variable for each month, could also set up beginning inventory as assistive variables as well, but it turns out to be more complicated to model for constraints
  ending_inventory_list = [pl.LpVariable(name=f'ending_inventory_{str(x)}', lowBound=0) for x in commodity_price_table.columns]

  # Objective function
  total_cash_inflow_from_sale = sum([sale_price * wheat_to_sell for sale_price, wheat_to_sell in zip(commodity_price_table.loc['Selling Price', :].to_list(), commodity_to_sell_list)])
  total_cash_outflow_to_buy = sum([purchase_price * wheat_to_buy for purchase_price, wheat_to_buy in zip(commodity_price_table.loc['Purchase Price', :].to_list(), commodity_to_buy_list)])
  total_cash_outflow_to_hold_wheat = sum([float(holding_cost_as_percent_to_purchase_price) * purchase_price * wheat_to_hold for purchase_price, wheat_to_hold in zip(commodity_price_table.loc['Purchase Price', :].to_list()[1:], ending_inventory_list[:-1])]
                                         + [float(holding_cost_as_percent_to_purchase_price) * beginning_inventory * commodity_price_table.loc['Purchase Price', :].to_list()[0]])
  commodity_buy_sell_model += total_cash_inflow_from_sale - total_cash_outflow_to_buy - total_cash_outflow_to_hold_wheat

  # constraint 1: wheat to sell in a given period cannot exceed the beginning inventory
  for x in range(0, len(commodity_price_table.columns)): 
    if x == 0:
      commodity_buy_sell_model += pl.LpConstraint(commodity_to_sell_list[x] - beginning_inventory , sense=pl.constants.LpConstraintLE, rhs = 0, name=f'sell<=beginning_inventory_{commodity_price_table.columns[x]}')
    else:
      commodity_buy_sell_model += pl.LpConstraint(commodity_to_sell_list[x] - ending_inventory_list[x-1], sense=pl.constants.LpConstraintLE, rhs = 0, name=f'sell<=beginning_inventory_{commodity_price_table.columns[x]}')
  
  # constraint 2: wheat to buy in a given period cannot exceed the warehouse size
  for x in range(0, len(commodity_price_table.columns)): 
    commodity_buy_sell_model += pl.LpConstraint(commodity_to_buy_list[x] , sense=pl.constants.LpConstraintLE, rhs = warehouse_size, name=f'buy<=warehousesize_{commodity_price_table.columns[x]}')

  # constraint 3: wheat to buy, together with the beginning inventory, needs to satisfy demand of that period
  for x in range(0, len(commodity_price_table.columns)): 
    if x == 0:
      commodity_buy_sell_model += pl.LpConstraint(beginning_inventory + commodity_to_buy_list[x] - demand[x], sense=pl.constants.LpConstraintGE, rhs = 0, name=f'buy<=begin+demand_{commodity_price_table.columns[x]}')
    else: 
      commodity_buy_sell_model += pl.LpConstraint(ending_inventory_list[x-1] + commodity_to_buy_list[x] - demand[x], sense=pl.constants.LpConstraintGE, rhs = 0, name=f'buy<=begin+demand_{commodity_price_table.columns[x]}')

  # constraint 4: inventory relationship
  for x in range(0, len(commodity_price_table.columns)): 
    if x == 0:
      commodity_buy_sell_model += pl.LpConstraint(ending_inventory_list[x] + commodity_to_sell_list[x] + demand[x] - commodity_to_buy_list[x], sense=pl.constants.LpConstraintEQ, rhs=beginning_inventory, name=f'inventory_relationship_{commodity_price_table.columns[x]}')
    else:
      commodity_buy_sell_model += pl.LpConstraint(ending_inventory_list[x] + commodity_to_sell_list[x] + demand[x] - commodity_to_buy_list[x] - ending_inventory_list[x-1], sense=pl.constants.LpConstraintEQ, rhs=0, name=f'inventory_relationship_{commodity_price_table.columns[x]}')

  # constraint 5: inventory balance must be <= warehouse size 
  for x in range(0, len(commodity_price_table.columns)): 
    commodity_buy_sell_model += pl.LpConstraint(ending_inventory_list[x], sense=pl.constants.LpConstraintLE, rhs = warehouse_size, name=f'ending_inventory<=warehousesize_{commodity_price_table.columns[x]}')
  return commodity_buy_sell_model


In [ ]:
demand_list = [7500, 8500, 16350, 17500, 4500, 6000, 7000, 8500, 14000, 15000]
commodity_buy_sell_model = setup_optimization_model(warehouse_size=20000, beginning_inventory=6000, commodity_name="wheat", commodity_price_table = commodity_price_table, demand = demand_list, holding_cost_as_percent_to_purchase_price = 0.1)

In [ ]:
commodity_buy_sell_model

Commodity_Supply_Management_Schedule:
MAXIMIZE
-0.025900000000000003*ending_inventory_2022_December + -0.02287*ending_inventory_2022_July + -0.02356*ending_inventory_2022_September + -0.02042*ending_inventory_2023_December + -0.022500000000000003*ending_inventory_2023_July + -0.023880000000000002*ending_inventory_2023_March + -0.0222*ending_inventory_2023_May + -0.02271*ending_inventory_2023_September + -0.01837*ending_inventory_2024_March + -0.2356*wheat_to_buy_2022_December + -0.2256*wheat_to_buy_2022_July + -0.2287*wheat_to_buy_2022_September + -0.2271*wheat_to_buy_2023_December + -0.222*wheat_to_buy_2023_July + -0.259*wheat_to_buy_2023_March + -0.2388*wheat_to_buy_2023_May + -0.225*wheat_to_buy_2023_September + -0.1837*wheat_to_buy_2024_July + -0.2042*wheat_to_buy_2024_March + 0.2259*wheat_to_sell_2022_December + 0.22325*wheat_to_sell_2022_July + 0.2253*wheat_to_sell_2022_September + 0.20305*wheat_to_sell_2023_December + 0.21*wheat_to_sell_2023_July + 0.2284*wheat_to_sell_2023_Marc

## Optimization and Results

In [ ]:
def model_optimize(model):
  status = model.solve()
  print("Model Status:{}".format(pl.LpStatus[commodity_buy_sell_model.status]))
  print("Maximum_cashflow =", model.objective.value())

  variable_dict = {'Name': [], 'Final_Value': [], 'Reduced_Cost': [], 'Low_Bound': [], 'Up_Bound': []}
  for variable in commodity_buy_sell_model.variables():
    variable_dict['Name'].append(variable.name)
    variable_dict['Final_Value'].append(variable.varValue)
    variable_dict['Reduced_Cost'].append(variable.dj)
    variable_dict['Low_Bound'].append(variable.lowBound)
    variable_dict['Up_Bound'].append(variable.upBound)
  variable_sensitivity = pd.DataFrame(variable_dict)

  constraints_info = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} for name, c in commodity_buy_sell_model.constraints.items()]
  constraint_sensitivity = pd.DataFrame(constraints_info)
  constraint_sensitivity

  return variable_sensitivity, constraint_sensitivity

In [ ]:
variable_sensitivity, constraint_sensitivity = model_optimize(model=commodity_buy_sell_model)

Model Status:Optimal
Maximum_cashflow = -22328.52


On surface, this doesn't sound vary appealing. After all, with all of the buys and sells planned, the mill/factory only ends up with a huge net cashoutflow. Unfortunately, this might just have to be the case. 

Here is one way to look at this. 

Notice the time when the future prices are fetched, March 2022, during Russian-Ukraine war. If anything should've caught the eye even before the linear model is configured and run, that is the big backwardation in the wheat futures prices. In a normal economic environment, the price the commodity future should follow a upward trend, or contango. With the war crisis evolving into a supply crisis, in the near term, wheat prices will be higher than the equivalents in the more distant future. Therefore, the wheat that the business has to buy will just needs to suffice the demand, together with beginning inventory here. With the holding costs considered, 0 inventory policy is further confirmed when we look at all of the ending inventories will be just 0. With a decreasing price into the future, plus the holding costs over time, there is really no point in buying extra, holding it and waiting for selling it at a higher price later. However, that said, this is abnormal environment, it doesn't mean there is no place for this model to exist anymore.  

In [ ]:
variable_sensitivity

,Name,Final_Value,Reduced_Cost,Low_Bound,Up_Bound
0,ending_inventory_2022_December,0.0,-0.00250,0,None
1,ending_inventory_2022_July,0.0,0.00000,0,None
2,ending_inventory_2022_September,0.0,-0.01666,0,None
3,ending_inventory_2023_December,0.0,-0.04332,0,None
4,ending_inventory_2023_July,0.0,-0.01950,0,None
5,ending_inventory_2023_March,0.0,-0.04408,0,None
6,ending_inventory_2023_May,0.0,-0.03900,0,None
7,ending_inventory_2023_September,0.0,-0.02061,0,None
8,ending_inventory_2024_July,0.0,-0.16220,0,None
9,ending_inventory_2024_March,0.0,-0.03887,0,None


**Action Plan**

All the variables with a name starts with "wheat" have their "final value" to indicate the transactional amount of wheat (number of bushels to buy or sell) should be in place at the futures contract delivery point. 

------

**Sensitivity Analysis**

Some important insights can be further drawn from the sensitivity analysis above. 

"Reduced costs" is important when assessing how much some price factors have to change before they can be included as part of the "final values" for a different optimal outcome. 

All reduced costs for "ending_inventory" variables indicate how much the holding costs has to lower before we should start building inventory. Echoing to the backwardation phenomenon, all the reduced costs for all transaction points considered here will essentially forces the holding costs to become "holding profits", which is just not possible. Therefore, there is no point indeed to hold any inventory, use/sell if you can is the story.

All reduced costs for "wheat_to_buy" series of variables are 0, since they are part of the optimal solution and indeed we are buying the minimum just to satisfy all the demands here. Nothing needs to "better".

All reduced costs for "wheat_to_sell" series of variables, most likely these reflect the bid-ask spread between buy and sell futures prices. In other words, when the spread closes up to 0, the decision variables will start changing. For a month where there is inventory to end with, in that month to sell one more unit, it takes a lot more: first, it needs to address the holding cost from inventory incurred in previous month; next, there needs to be higher selling price to close up the bid-ask spread just like any other delivery month, and finally the selling price needs further to account for the opportunity cost not selling in the previous month. For most months above, again, because we are looking at a steady backwardation trend on the wheat price, most of the reduced prices simply reflects the bid-ask spread.

In [ ]:
constraint_sensitivity

,name,shadow price,slack
0,sell<=beginning_inventory_2022_July,-0.00000,6000.0
1,sell<=beginning_inventory_2022_September,0.01742,-0.0
2,sell<=beginning_inventory_2022_December,-0.00000,-0.0
3,sell<=beginning_inventory_2023_March,-0.00000,-0.0
4,sell<=beginning_inventory_2023_May,-0.00000,-0.0
5,sell<=beginning_inventory_2023_July,-0.00000,-0.0
6,sell<=beginning_inventory_2023_September,-0.00000,-0.0
7,sell<=beginning_inventory_2023_December,-0.00000,-0.0
8,sell<=beginning_inventory_2024_March,-0.00000,-0.0
9,sell<=beginning_inventory_2024_July,-0.00000,-0.0


On the constraint side, 2 highly valuable groups of insights reside with shadow prices and slack. Slacks only show up when a constraint becomes non-binding, in other words, not used up. 

Shadow prices behave the opposite; they only have non-zero values when the constraint is binding, or used up. Essentially, what shadow price can tell how much a constraint is worth to us to the extent that we are willing to relax such constraint by 1 unit. All shadow prices are interpretable, i.e. they have specific contextual meaning, but not all necessarily all that useful for decision-making. Take a look at the "inventory_relationship" series, which is to state that if we want to have one more bushel at the end of that delivery time, we need to pay for it at the purchase price then. It is interpretable, but not that meaningful. However, some more meaningful series are "buy<=begin+demand" series, these shadow prices tell the bid-ask gap or how much the purchase has to get better before we can consider allowing to buy more. Also, it indicates that this is the time where we are maxing out on buying opportunities.